## Information Coding 2023-24

## Lab assignment 5 (Lab session 6, 21-12-2023)

## Public key cryptography

 (Estimated working time: 2 hours lab + 3 hours home);
	
 Student names:
 Theodor Ioan Buliga
 Javier Cirugeda Bugallo
 

 INSTRUCTIONS
  Practices are to be done in groups of two (or one). 
  This solved practice has to be uploaded in Moodle before November 28th, 23:55h. 
  The submission must be done by the first author (alphabetical order).
    Name the file with authors' names.
  This practice accounts for 5% of final grade for both autors.
  Grading items:
    * Defined functions 50% (eficiency, clarity, code documentation)
    * Exercise solving 40%
    * Mathematical correction, clarity of results, language precission, 10%.
  To get any credit, physical presence in the practice session is a must.


 

### 1 (1 pt) Bit average and entropy

#### 1.1 a)
Define a function that calculates the bit average of a code.
	
Input:
- A list (A) of pairs, each formed by a symbol of an alphabet (Alf) and a probability or frequence
- A valid code scheme C, with source alphabet Alf
	

#### 1.2 b)
Define a procedure that calculates the entropy of a source of information.

Input:
- A list (A) of pairs, each formed by a symbol of an alphabet and a probability or frequence


####  1.3 
Try the functions above with the following data, and comment the results

In [107]:
from math import log2

def bit_average(A: list, C: list) -> float:
    """
    Calculate the bit average of a source A using a code C.

    Parameters:
    - A (list): List of tuples representing symbols and their probabilities.
    - C (list): List of tuples representing symbols and their corresponding codes.

    Returns:
    - float: Bit average of source A using code C.
    """
    # Check if A and C are lists
    if type(A) != list or type(C) != list:
        raise TypeError("A and C must be lists")

    total = 0

    # Convert code C to a dictionary for faster lookup
    C = dict(C)

    # Calculate the bit average using the provided code
    for symbol, prob in A:
        # Check if the symbol is present in the code
        if symbol not in C:
            raise ValueError("Error: symbol not in C")

        # Add the product of probability and code length to the total
        total += prob * len(C[symbol])

    # Round the result to 2 decimal places
    return round(total, 2)


def entropy(A: list) -> float:
    """
    Calculate the entropy of a source A.

    Parameters:
    - A (list): List of tuples representing symbols and their probabilities.

    Returns:
    - float: Entropy of source A.
    """
    # Check if A is a list
    if type(A) != list:
        raise TypeError("Error: A is not a list")

    entropy = 0

    # Calculate the entropy using the formula -p*log2(p)
    for _, prob in A:
        entropy += prob * log2(1/prob)

    # Round the result to 2 decimal places
    return round(entropy, 2)

# Test data
Afr = [["a", 0.2], ["b", 0.3], ["c", 0.2], ["d", 0.15], ["e", 0.15]]
C = [["a", "11"], ["b", "0"], ["c", "1010"], ["d", "1011"], ["e", "100"]]

# Calculate bit average
ba = bit_average(Afr, C)
print("Bit Average:", ba)

# Calculate entropy
ent = entropy(Afr)
print("Entropy:", ent)


Bit Average: 2.55
Entropy: 2.27


Comments:

###  2 Huffman algorithm
We will now program the Huffman algorithm (see the algorithm 1.5.2 in the notes)
	
Name of the procedure must be Huffman
	
Input is a list (A) of pairs, each formed by a symbol of an alphabet and a probability or frequence.
Output is a code scheme (of an optimal prefix code) 
	
To apply the algorithm we need a local variable (a forest B), a set or list of pairs.

The first element of each pair is a list of pairs, [char,codeword],  (i.e. a tree);
the second element is the corresponding weight.
	
We initialize the forest with each vertex encoded as an empty codeword (""), with weight given by the corresponding frequency.
	
The algorithm then goes:
while B has more than one element do
- Choose two trees with minimal weight (w(p)>=w(q)).
- In case of a tie, choose the tree with less leaves, or less depth.
- Create a new tree t, having p as left subtree and q as right subtree.
     (left branch from root labelled with 0, right branch with 1)
- Assign the weight to the new tree as the sum of weights of p and q
- Update B: delete p and q, add t.
	
Output: the only element of B
	
	


In [38]:
### 2.1 Hints
#For the source Afr in the exercise above, the forest B would initialize as:
B=[]
for sym in Afr:
    B=B+[[[sym[0],""], sym[1]]]
print(B)

#Let us now see how to combine two minimal weight trees p, q into a new tree t. 
#For instance, suppose the two minimal weight trees are: 
p = [ [["d","1"],["e","0"]] , .3]
q = [ [["a","01"]] , .2]
# The new tree has p as left subtree (adding an initial "0" to all codewords in p, as the new branch is labelled "0"),
# and q as right subtree (adding an initial "1" to all codewords).
#The new weight is the sum of weights of p and q: 
t1 = []
for i in range(0,len(p[0])):
    t1 = t1 + [ [p[0][i][0],"0"+p[0][i][1]] ]
for j in range(0,len(q[0])):
    t1 = t1 + [ [q[0][j][0],"1"+q[0][j][1]] ]
t = [ t1, p[1]+q[1] ]
print(t)

[[['a', ''], 0.2], [['b', ''], 0.3], [['c', ''], 0.2], [['d', ''], 0.15], [['e', ''], 0.15]]
[[['d', '01'], ['e', '00'], ['a', '101']], 0.5]


### 2.2 (1 pt) Minimal subtree selection
First, write a function to select the minimal weight tree from a forest.
- Input: list of pairs: The first element of each pair is a list of pairs, [char,codeword],  (i.e. a tree);
    the second element is the corresponding weight.
    In case of equal weights, the minimal order tree selected is the one with less leaves or with less depth.
    (this is made in order to obtain the minimal variance Huffman code)
- Output: pair with minimal weight.

In [39]:
def minimalWeight(forest):
    """
    Find the tree with the minimal weight in the given forest.

    Parameters:
    - forest (list): List of trees represented as tuples. Each tuple contains a tree (list),
                    its weight (numeric value), and its depth (numeric value).

    Returns:
    - tuple: The tree with the minimal weight, considering ties based on the number of leaves and depth.
    """
    # Sort the list by weight, then by the number of leaves, and finally by depth
    forest.sort(key=lambda x: (x[1], len(x[0]), x[2]))

    # Return the smallest element (tree with minimal weight) of the sorted forest
    return forest[0]

print(minimalWeight([[[['a', '']], 0.2], [[['b', '']], 0.3], [[['c', '']], 0.2], [[['d', '']], 0.15], [[['e', '']], 0.15]]))

[[['d', '']], 0.15]


###  2.3 (2 pt) Huffman algorithm
Write a function Huffman implementing the Huffman algorithm.

Input:
- A list (A) of pairs, each formed by a symbol of an alphabet (Alf) and a probability or frequence

Output:
- Code scheme obtained by Huffman algorithm.

Notes:
- list1+list2 gives as a result the concatenation of both lists.
- We can delete an element e of a list l using l.delete(e)
	

In [108]:
def Huffman(A):
    """
    Generate a Huffman tree based on the given symbol frequencies.

    Parameters:
    - A (list): List of tuples representing symbols and their frequencies.

    Returns:
    - list: The root of the Huffman tree, represented as nested lists with symbols and their corresponding codes.
    """
    # Initialize the forest with individual trees for each symbol
    B = [[[ [sym[0], ""] ], sym[1]] for sym in A]

    # Continue merging trees until only one tree remains in the forest
    while len(B) > 1:
        # Find and remove the tree with the smallest weight from the forest
        p = minimalWeight(B)
        B.remove(p)
        # Find and remove the tree with the second smallest weight from the forest
        q = minimalWeight(B)
        B.remove(q)

        # Create a new tree by combining the two smallest trees and add it back to the forest
        t1 = [[p[0][i][0], "0" + p[0][i][1]] for i in range(len(p[0]))]
        t1 += [[q[0][j][0], "1" + q[0][j][1]] for j in range(len(q[0]))]
        t = [t1, p[1] + q[1]]
        B.append(t)

    # Return the root of the Huffman tree (the remaining tree in the forest)
    return B[0][0]


print(Huffman([["a",0.2],["b",0.1],["c",0.15],["d",0.25], ["e",0.3]]))

[['a', '00'], ['d', '01'], ['b', '100'], ['c', '101'], ['e', '11']]


### 2.4 (1 pt) Exercises
(Problem 6.1) Apply the function to obtain Huffman encodings for the next sources:
Calculate the bit average and entropy in each case.

In [45]:
Alfa1 = [["a",0.2],["b",0.1],["c",0.15],["d",0.25], ["e",0.3]];
Alfa2 = [["a",0.1],["b",0.25],["c",0.05],["d",0.15], ["e",0.3],  ["f",0.07], ["g",0.08]];

huffman = Huffman(Alfa1)
print("Huffman encoding:\n", huffman)
print("Bit average:\n", bit_average(Alfa1, huffman))
print("Entropy:\n", entropy(Alfa1))


print("\n----------------------------------------------\n")

huffman = Huffman(Alfa2)
print("Huffman encoding:\n", huffman)
print("Bit average:\n", bit_average(Alfa2, huffman))
print("Entropy:\n", entropy(Alfa2))



Huffman encoding:
 [['a', '00'], ['d', '01'], ['b', '100'], ['c', '101'], ['e', '11']]
Bit average:
 2.25
Entropy:
 2.23

----------------------------------------------

Huffman encoding:
 [['g', '000'], ['a', '001'], ['b', '01'], ['c', '1000'], ['f', '1001'], ['d', '101'], ['e', '11']]
Bit average:
 2.57
Entropy:
 2.54


Solve the Problem 6.3.


In [109]:
message = "Las matematicas son a la verdad lo que audrey hepburn es a marilyn monroe"

def convertToProbList(message: str):
    """
    Convert a message string into a list of symbol probabilities.

    Parameters:
    - message (str): Input message string.

    Returns:
    - list: List of tuples, each containing a symbol and its probability in the message.
    """
    # Define an alphabet
    ALFA = "abcdefghijklmnopqrstuwwvxyz"
    length = len(message)
    res = []

    # Iterate through characters in the alphabet
    for character in ALFA:
        # Check if the character is present in the message
        if character in message.lower():
            # Count the occurrences of the character in the message
            occ = message.lower().count(character)
            # Calculate the probability of the character and add it to the result list
            res.append([character, occ / length])

    return res
            

convertedMessage = convertToProbList(message)
huffman = Huffman(convertedMessage)
print("Huffman encoding:\n", huffman)
print("Entropy:\n", entropy(convertedMessage))
print("Bit average:\n", bit_average(convertedMessage, huffman))

Huffman encoding:
 [['r', '000'], ['d', '0010'], ['u', '0011'], ['e', '010'], ['l', '0110'], ['m', '0111'], ['n', '1000'], ['o', '1001'], ['s', '1010'], ['i', '10110'], ['t', '10111'], ['y', '11000'], ['b', '110010'], ['c', '110011'], ['h', '110100'], ['p', '110101'], ['q', '110110'], ['v', '110111'], ['a', '111']]
Entropy:
 3.44
Bit average:
 3.23


### 3 CRC: Strings and polynomials (1.5 pts)
### 3.1 Polynomial to strings
Define a function that gives the bitstring associated to a polynomial of Z2(x).
	
    Input:
    - A polynomial in the indeterminate x
	
    Output:
    - The bitstring of coefficients


In [4]:
# Hints 
from sympy import *

x= symbols('x')
p= Poly(x**3+x**2+1,x,modulus=2)
print(p)
l=p.all_coeffs()
n=p.degree()
print(l)
print(n)

Poly(x**3 + x**2 + 1, x, modulus=2)
[1, 1, 0, 1]
3


In [111]:
def polynomial_to_bitstring(poly):
    """
    Convert a polynomial to a bitstring representation based on its coefficients.

    Parameters:
    - poly: Polynomial object representing the polynomial.

    Returns:
    - str: Bitstring representation of the polynomial coefficients.
    """
    # Get the coefficients of the polynomial
    coefficients = poly.all_coeffs()

    # Convert the coefficients to a bitstring
    bitstring = ''.join(map(str, coefficients))

    return bitstring


x = symbols('x')
polynomial = Poly(x**3+x**2+1,x,modulus=2)
bitstring = polynomial_to_bitstring(polynomial)
x= symbols('x')
print(polynomial)
l=polynomial.all_coeffs()
n=polynomial.degree()
print(l)
print(n)


print(f"Polynomial: {polynomial}")
print(f"Bitstring: {bitstring}")

Poly(x**3 + x**2 + 1, x, modulus=2)
[1, 1, 0, 1]
3
Polynomial: Poly(x**3 + x**2 + 1, x, modulus=2)
Bitstring: 1101


### 3.2 Strings to polynomials
Define a function that gives the polynomial of Z2(x) associated to a bitstring.
	
    Input:
    - A bitstring of 0s and 1s, an an-1 ... a1 a0
	
    Output:
    - The polynomial an*x^n+an-1*x^(n-1)+...+a1*x+a0

Hints: - you can use a conditional to transform chars into integers

In [110]:
def stringToPolynomial(s: str):
    """
    Convert a bitstring to a polynomial representation.

    Parameters:
    - s (str): Input bitstring.

    Returns:
    - Poly: Polynomial representation of the input bitstring.
    """
    res = ""

    # Iterate through the bitstring and create a polynomial out of it
    for i in range(len(s)):
        # If it is the last character in the bitstring, do not add the plus sign at the end
        if i == len(s) - 1:
            res += s[-1]
            break
        elif s[i] == '0':
            continue
        # Create a string with 'x**' + the exponent
        res += "x**" + str(len(s) - i - 1) + "+"
    
    # Convert the resulting string to a polynomial
    return Poly(res, x, modulus=2)


poly = stringToPolynomial("101010")
print(poly)
print(poly.all_coeffs())
print(poly.degree())

Poly(x**5 + x**3 + x, x, modulus=2)
[1, 0, 1, 0, 1, 0]
5


### 3.3 Examples

To perform calculations in Z2[x]:
 

In [6]:
g = Poly (x**4+x**2+x+1,x,modulus=2)
p = Poly (x**14+1,x,modulus=2)
r=div(p,g)[1]
print(r) #that is the remainder 

Poly(0, x, modulus=2)


Compute the remainder of "1001010010010" divided by "1010010", written as a bitstring

In [52]:
# Example usage
dividend = "1001010010010"
divisor = "1010010"

poly1 = stringToPolynomial(dividend)
poly2 = stringToPolynomial(divisor)

remindPoly = div(poly1, poly2)[1]
print("Reminder:", remindPoly)
print("Reminder as a bitstring:", polynomial_to_bitstring(remindPoly))

Reminder: Poly(x**4 + x**3 + x**2, x, modulus=2)
Reminder as a bitstring: 11100


### 4 CRC systematic encoding and decoding
###  4.1 Generator polynomial (0.5 pts)

Write a function to check if a bitstring g is associated to
	a generator g(x) of a CRC of length n.

    Input: A bitstring g, an integer n
	
    Output: A boolean: is g a valid generator?

Hint: It suffices to check that g(x) divides x^n+1.
	Note that deg g(x)=len(g)-1=n-k
 

In [114]:
def is_valid_generator(g, n):
    
    x = symbols('x')

    # Construct the polynomial x^n + 1
    xn_plus_1 = Poly(x**n + 1, x)

    # Convert the bitstring g to a list of coefficients
    g_coeffs = [int(bit) for bit in g]

    # Construct the polynomial g(x)
    gx = Poly(sum(c * x**i for i, c in enumerate(reversed(g_coeffs))), x)

    # Polynomial division: Check if g(x) divides x^n + 1
    _, remainder = divmod(xn_plus_1, gx)

    # If the remainder is zero, g(x) divides x^n + 1
    return remainder.is_zero


# Example usage
generator_bitstring = "1001"
crc_length = 4

result = is_valid_generator(generator_bitstring, crc_length)
print(f"Is '{generator_bitstring}' a valid generator for CRC-{crc_length}? {result}")

Is '101' a valid generator for CRC-5? False


### 4.2 Systematic encoding (1.5 pts)
Write a function CRCEncode to compute the systematic encoding of a source word u
	in a CRC code of length n with generator g.
	
    Input:
    - a bitstring of length k, the source word u
	- a bitstring of length n-k+1, the generator g
	- an integer n
	
    Output:
    - A bitstring of length n, the encoded word.
 

In [116]:
from math import log, ceil

def CRCEncode(dataword, generator, length):
    """
    Encode a dataword using a cyclic redundancy check (CRC) generator polynomial.

    Parameters:
    - dataword (str): Binary string representing the input dataword.
    - generator (str): Binary string representing the CRC generator polynomial.
    - length (int): Length of the input dataword.

    Returns:
    - str: Binary string representing the CRC encoded codeword.
    """
    # Check if the length of the dataword matches the specified length
    if len(dataword) != length:
        raise ValueError("Length error")

    # Convert dataword to an integer
    dword = int(dataword, 2)
    l_gen = len(generator)

    # Append 0s to the dividend
    dividend = dword << (l_gen - 1)

    # Shift specifies the number of least significant bits not being XORed
    shft = ceil(log(dividend + 1, 2)) - l_gen

    # Convert generator to an integer
    generator = int(generator, 2)

    # Perform CRC encoding
    while dividend >= generator or shft >= 0:
        # Bitwise XOR the MSBs of dividend with generator
        # Replace the operated MSBs from the dividend with the remainder generated
        rem = (dividend >> shft) ^ generator
        dividend = (dividend & ((1 << shft) - 1)) | (rem << shft)

        # Change shft variable
        shft = ceil(log(dividend + 1, 2)) - l_gen

    # Finally, AND the initial dividend with the remainder (=dividend)
    codeword = dword << (l_gen - 1) | dividend

    # Convert codeword to a binary string and remove the prefix
    return bin(codeword).lstrip("-0b")


# Driver code
dataword = "11010"
generator = "101"
CRCEncode(dataword, generator, 5)


'1101001'

### 4.3 Error detection and decoding (1 pt)
Write a function CRCDecode to check if a received word v is a code word
of a CRC of length n and generator g. If the answer is yes, give the
source word as output. In other case, return an empty string.

    Input:
    - a bitstring of length n, the received word v
	- a bitstring of length n-k+1, the generator g
	- an integer n
	
    Output:
    - If no error is detected, a bitstring of length k, the source word.
	- If an error is detected, an empty string.


In [117]:
from sympy import symbols, Poly

def CRCDecode(v, g, n):
    """
    Decode a received codeword using a cyclic redundancy check (CRC) generator polynomial.

    Parameters:
    - v (str): Binary string representing the received codeword.
    - g (str): Binary string representing the CRC generator polynomial.
    - n (int): Length of the received codeword.

    Returns:
    - str: Binary string representing the decoded source word if no error is detected, otherwise an empty string.
    """
    # Define the symbolic variable x for polynomial operations
    x = symbols('x')

    # Convert the bitstrings v and g to lists of integers
    v_coeffs = [int(bit) for bit in v]
    g_coeffs = [int(bit) for bit in g]

    # Construct the polynomials v(x) and g(x)
    vx = Poly(sum(c * x**i for i, c in enumerate(reversed(v_coeffs))), x)
    gx = Poly(sum(c * x**i for i, c in enumerate(reversed(g_coeffs))), x)

    # Perform polynomial division: divide vx by gx
    _, remainder = divmod(vx, gx)

    # If the remainder is zero, no error is detected
    if remainder.is_zero:
        # Extract the source word from the received word
        source_word = v[:len(v) - len(g) + 1]
        return source_word
    else:
        # Error is detected, return an empty string
        return ""


# Example usage
received_word = "1101001"
generator = "101"
crc_length = 5

result = CRCDecode(received_word, generator, crc_length)
if result:
    print(f"Source word: {result}")
else:
    print("Error detected, the received word is not a valid code word.")

Source word: 11010


In [106]:
# TESTS
dataword1 = "11010"
dataword2 = "10111"
dataword3 = "10101"
dataword4 = "11010"
generator = "101"
crc_length = 5

encoded_word1 = CRCEncode(dataword1, generator, crc_length)
encoded_word2 = CRCEncode(dataword2, generator, crc_length)
encoded_word3 = CRCEncode(dataword3, generator, crc_length)
encoded_word4 = CRCEncode(dataword4, generator, crc_length)

received_word1 = CRCDecode(encoded_word1, generator, crc_length)
received_word2 = CRCDecode(encoded_word2, generator, crc_length)
received_word3 = CRCDecode(encoded_word3, generator, crc_length)
received_word4 = CRCDecode(encoded_word4, generator, crc_length)

print(f"Sent: {dataword1} - Encoded: {encoded_word1} - Received: {received_word1}")
print(f"Sent: {dataword2} - Encoded: {encoded_word2} - Received: {received_word2}")
print(f"Sent: {dataword3} - Encoded: {encoded_word3} - Received: {received_word3}")
print(f"Sent: {dataword4} - Encoded: {encoded_word4} - Received: {received_word4}")

Sent: 11010 - Encoded: 1101001 - Received: 11010
Sent: 10111 - Encoded: 1011111 - Received: 10111
Sent: 10101 - Encoded: 1010101 - Received: 10101
Sent: 11010 - Encoded: 1101001 - Received: 11010


### 4.4 Exercise (0.5 pts)
Solve the exercise 6.8 in the notes.


# (a) Determine the code parameters nn and kk.

Given the generator polynomial g(x)=x^8+1, we can determine the code parameters:

    The code length nn is the degree of the generator polynomial, so n=8.
    The message length k is given by n−deg(g), so k=8−1=7.